# 🗄️ Notebook 05: Vector Stores

**LangChain 1.0.5+ | Mixed Level Class**

## 🎯 Objectives
1. Understand vector stores
2. Use InMemoryVectorStore (testing)
3. Use FAISS (production)
4. Use Chroma (persistent)
5. Compare vector stores

In [1]:
from dotenv import load_dotenv

In [2]:
from pathlib import Path
load_dotenv()
print("✅ Setup complete")

✅ Setup complete


## 1. What are Vector Stores? 🤔

### 🔰 BEGINNER

**Vector Stores** are databases optimized for finding similar vectors.

Like a library:
- Traditional database: "Find book with ISBN 12345"
- Vector store: "Find books similar to this topic"

They enable **semantic search**!

## 2. InMemoryVectorStore (Simple)

In [4]:
import os

In [ ]:
# Google Gemini Embeddings
if os.getenv("GOOGLE_API_KEY"):
    from langchain_google_genai import GoogleGenerativeAIEmbeddings
    
    embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    
    vector = embeddings.embed_query("Hello world")
    print(f"Gemini vector dimensions: {len(vector)}")
else:
    print("⚠️ Google API key not found. Set GOOGLE_API_KEY in .env")

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# Initialize
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = InMemoryVectorStore(embedding=embeddings)

# Add documents
docs = [
    Document(page_content="Python is a programming language"),
    Document(page_content="Machine learning uses algorithms"),
    Document(page_content="The sun is a star")
]

vector_store.add_documents(docs)
print(f"✅ Added {len(docs)} documents\n")

# Search
results = vector_store.similarity_search("What is Python?", k=2)

print("Search Results:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
    
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
    
#vector = embeddings.embed_query("Hello world")
vector_store = InMemoryVectorStore(embedding=embeddings)

# Initialize
#embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
#vector_store = InMemoryVectorStore(embedding=embeddings)

# Add documents
docs = [
    Document(page_content="Python is a programming language"),
    Document(page_content="Machine learning uses algorithms"),
    Document(page_content="The sun is a star")
]

vector_store.add_documents(docs)
print(f"✅ Added {len(docs)} documents\n")

# Search
results = vector_store.similarity_search("What is Python?", k=2)

print("Search Results:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")

✅ Added 3 documents

Search Results:
1. Python is a programming language
2. Machine learning uses algorithms


## 3. FAISS (Fast & Production-Ready)

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

# Load and split a PDF
pdf_path = "attention.pdf"

if Path(pdf_path).exists():
    print("Loading PDF...")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # Split
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")
    
    # Create FAISS vectorstore
    print("Creating embeddings (this may take a minute)...")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    
    # Save to disk
    vectorstore.save_local("./faiss_index_notebook")
    print("✅ FAISS index saved\n")
    
    # Search
    query = "What is attention mechanism?"
    results = vectorstore.similarity_search(query, k=3)
    
    print(f"Query: {query}\n")
    for i, doc in enumerate(results, 1):
        print(f"Result {i}:")
        print(f"  {doc.page_content[:150]}...\n")
else:
    print(f"❌ PDF not found: {pdf_path}")

Loading PDF...
Split into 52 chunks
Creating embeddings (this may take a minute)...
✅ FAISS index saved

Query: What is attention mechanism?

Result 1:
  3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and ...

Result 2:
  Scaled Dot-Product Attention
 Multi-Head Attention
Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several
att...

Result 3:
  The Transformer uses multi-head attention in three different ways:
• In "encoder-decoder attention" layers, the queries come from the previous decoder...



## 4. Loading Existing FAISS Index

In [8]:
# Load existing index
if Path("./faiss_index_notebook").exists():
    loaded_vectorstore = FAISS.load_local(
        "./faiss_index_notebook",
        embeddings,
        allow_dangerous_deserialization=True
    )
    print("✅ Loaded existing FAISS index")
    
    # Use it
    results = loaded_vectorstore.similarity_search("transformers", k=2)
    print(f"\nFound {len(results)} results")
else:
    print("No existing index found")

✅ Loaded existing FAISS index

Found 2 results


## 5. Chroma (Persistent & Easy)

In [6]:
!uv pip install langchain-chroma

Resolved 93 packages in 914ms                                        
⠙ Preparing packages... (0/14)                                                  
⠙ Preparing packages... (0/14)------------     0 B/70.83 KiB            
⠙ Preparing packages... (0/14)------------ 14.84 KiB/70.83 KiB          
opentelemetry-proto ------------------------------ 14.84 KiB/70.83 KiB
⠙ Preparing packages... (0/14)----------     0 B/129.25 KiB             
opentelemetry-proto ------------------------------ 14.84 KiB/70.83 KiB
⠙ Preparing packages... (0/14)---------- 14.84 KiB/129.25 KiB           
typer      ------------------------------     0 B/45.93 KiB
opentelemetry-proto ------------------------------ 14.84 KiB/70.83 KiB
⠙ Preparing packages... (0/14)---------- 14.84 KiB/129.25 KiB           
typer      ------------------------------ 14.89 KiB/45.93 KiB
opentelemetry-proto ------------------------------ 14.84 KiB/70.83 KiB
⠙ Preparing packages... (0/14)---------- 14.84 KiB/129.25 KiB           
typer

In [9]:
from langchain_chroma import Chroma

# Create Chroma vectorstore
chroma_store = Chroma(
    collection_name="my_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_db_notebook"
)

# Add documents
sample_docs = [
    Document(
        page_content="RAG combines retrieval and generation",
        metadata={"topic": "rag", "difficulty": "intermediate"}
    ),
    Document(
        page_content="LangChain simplifies LLM applications",
        metadata={"topic": "langchain", "difficulty": "beginner"}
    )
]

chroma_store.add_documents(sample_docs)
print("✅ Added documents to Chroma\n")

# Search with metadata filter
results = chroma_store.similarity_search(
    "Tell me about RAG",
    k=2,
    filter={"topic": "rag"}
)

print("Filtered search results:")
for doc in results:
    print(f"  {doc.page_content}")
    print(f"  Metadata: {doc.metadata}")

✅ Added documents to Chroma

Filtered search results:
  RAG combines retrieval and generation
  Metadata: {'difficulty': 'intermediate', 'topic': 'rag'}


## 6. Vector Store Comparison

| Store | Persistent | Speed | Metadata Filtering | Best For |
|-------|-----------|-------|-------------------|----------|
| **InMemory** | ❌ | ⚡⚡⚡ | ❌ | Testing |
| **FAISS** | ✅ | ⚡⚡⚡ | Limited | Production, Speed |
| **Chroma** | ✅ | ⚡⚡ | ✅ | Development, Filtering |

## Summary

✅ Vector stores enable semantic search
✅ InMemory for testing
✅ FAISS for speed and scale
✅ Chroma for metadata filtering
✅ Always persist for production!

**Next:** Retrieval Strategies!